# SPR 2026 - BERTimbau v5 (Cross-Validation Thresholds)

**Baseado em:** BERTimbau v4 (0.82073) 🏆

**Melhoria:** Usar cross-validation para encontrar thresholds mais estáveis

**Estratégia:**
- 5-Fold CV para treinar modelo
- Calcular thresholds ótimos em cada fold
- Usar média dos thresholds para submissão

---
## CONFIGURAÇÃO KAGGLE:
1. **Add Input** → **Models** → `bertimbau-ptbr-complete`
2. **Add Input** → **Competition** → `spr-2026-mammography-report-classification`
3. **Settings** → Internet → **OFF**, GPU → **T4 x2**
---

In [ ]:
# ===== BERTIMBAU v5 - CV THRESHOLDS =====

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - BERTimbau v5 (CV Thresholds)")
print("="*60)

# ===== CONFIG =====
SEED = 42
MAX_LEN = 256
BATCH_SIZE = 8
EPOCHS = 3
LR = 2e-5
NUM_CLASSES = 7
N_FOLDS = 5
FOCAL_GAMMA = 2.0
FOCAL_ALPHA = 0.25

DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(SEED)
np.random.seed(SEED)

def find_model_path():
    base = '/kaggle/input'
    def search_dir(directory, depth=0, max_depth=10):
        if depth > max_depth: return None
        try:
            for item in os.listdir(directory):
                path = os.path.join(directory, item)
                if os.path.isdir(path) and os.path.exists(os.path.join(path, 'config.json')):
                    return path
                result = search_dir(path, depth + 1, max_depth) if os.path.isdir(path) else None
                if result: return result
        except: pass
        return None
    return search_dir(base)

MODEL_PATH = find_model_path()
print(f"Device: {device} | Model: {MODEL_PATH}")

In [ ]:
# ===== FOCAL LOSS & DATASET =====
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0):
        super().__init__()
        self.alpha, self.gamma = alpha, gamma
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        return (self.alpha * (1 - pt) ** self.gamma * ce_loss).mean()

class TextDataset(Dataset):
    def __init__(self, texts, labels=None, tokenizer=None, max_len=256):
        self.texts, self.labels, self.tokenizer, self.max_len = texts, labels, tokenizer, max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tokenizer(str(self.texts[idx]), truncation=True, max_length=self.max_len, padding='max_length', return_tensors='pt')
        item = {'input_ids': enc['input_ids'].squeeze(), 'attention_mask': enc['attention_mask'].squeeze()}
        if self.labels is not None: item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
test_ds = TextDataset(test_df['report'].values, None, tokenizer, MAX_LEN)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)
print(f"Train: {len(train_df)}, Test: {len(test_df)}")

In [ ]:
# ===== CROSS-VALIDATION =====
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=SEED)
fold_thresholds = []
fold_f1s = []
oof_probs = np.zeros((len(train_df), NUM_CLASSES))
test_probs_sum = np.zeros((len(test_df), NUM_CLASSES))

def find_optimal_thresholds(probs, labels):
    thresholds = {}
    for c in range(NUM_CLASSES):
        best_thresh, best_f1 = 0.5, 0
        for thresh in np.arange(0.1, 0.9, 0.05):
            preds = np.argmax(probs, axis=1)
            for i in range(len(probs)):
                if probs[i, c] >= thresh and probs[i, c] == probs[i].max():
                    preds[i] = c
            f1 = f1_score(labels, preds, average='macro')
            if f1 > best_f1:
                best_f1, best_thresh = f1, thresh
        thresholds[c] = best_thresh
    return thresholds

for fold, (train_idx, val_idx) in enumerate(skf.split(train_df['report'], train_df['target'])):
    print(f"\n{'='*60}")
    print(f"FOLD {fold+1}/{N_FOLDS}")
    print("="*60)
    
    train_texts = train_df['report'].iloc[train_idx].values
    val_texts = train_df['report'].iloc[val_idx].values
    train_labels = train_df['target'].iloc[train_idx].values
    val_labels = train_df['target'].iloc[val_idx].values
    
    train_ds = TextDataset(train_texts, train_labels, tokenizer, MAX_LEN)
    val_ds = TextDataset(val_texts, val_labels, tokenizer, MAX_LEN)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
    
    torch.manual_seed(SEED)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, num_labels=NUM_CLASSES, local_files_only=True)
    model.to(device)
    criterion = FocalLoss(alpha=FOCAL_ALPHA, gamma=FOCAL_GAMMA)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader)*EPOCHS)
    
    for epoch in range(EPOCHS):
        model.train()
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            optimizer.zero_grad()
            outputs = model(batch['input_ids'].to(device), attention_mask=batch['attention_mask'].to(device))
            loss = criterion(outputs.logits, batch['labels'].to(device))
            loss.backward()
            optimizer.step()
            scheduler.step()
    
    # Avaliar fold
    model.eval()
    val_probs_list = []
    with torch.no_grad():
        for batch in val_loader:
            outputs = model(batch['input_ids'].to(device), attention_mask=batch['attention_mask'].to(device))
            val_probs_list.append(F.softmax(outputs.logits, dim=-1).cpu().numpy())
    
    val_probs = np.vstack(val_probs_list)
    oof_probs[val_idx] = val_probs
    
    # Calcular thresholds do fold
    fold_thresh = find_optimal_thresholds(val_probs, val_labels)
    fold_thresholds.append(fold_thresh)
    
    preds = np.argmax(val_probs, axis=1)
    f1 = f1_score(val_labels, preds, average='macro')
    fold_f1s.append(f1)
    print(f"Fold {fold+1} F1: {f1:.5f}")
    print(f"Thresholds: {fold_thresh}")
    
    # Acumular predições de teste
    with torch.no_grad():
        for i, batch in enumerate(test_loader):
            outputs = model(batch['input_ids'].to(device), attention_mask=batch['attention_mask'].to(device))
            start = i * BATCH_SIZE
            end = min(start + BATCH_SIZE, len(test_df))
            test_probs_sum[start:end] += F.softmax(outputs.logits, dim=-1).cpu().numpy()

test_probs_avg = test_probs_sum / N_FOLDS

In [ ]:
# ===== THRESHOLDS MÉDIOS E SUBMISSION =====
print("\n" + "="*60)
print("RESULTADOS CV")
print("="*60)
print(f"F1-Macro por fold: {fold_f1s}")
print(f"F1-Macro médio: {np.mean(fold_f1s):.5f} ± {np.std(fold_f1s):.5f}")

# Média dos thresholds
avg_thresholds = {}
for c in range(NUM_CLASSES):
    avg_thresholds[c] = round(np.mean([ft[c] for ft in fold_thresholds]), 2)
print(f"\nThresholds médios: {avg_thresholds}")

def apply_thresholds(probs, thresholds):
    preds = []
    for i in range(len(probs)):
        pred = np.argmax(probs[i])
        for c in [6, 5]:
            if probs[i, c] >= thresholds[c]:
                pred = c
                break
        preds.append(pred)
    return np.array(preds)

predictions = apply_thresholds(test_probs_avg, avg_thresholds)
submission = pd.DataFrame({'ID': test_df['ID'], 'target': predictions})
submission.to_csv('submission.csv', index=False)
print(f"\nSubmission salva!")
print(submission['target'].value_counts().sort_index())